In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from utils import crop_image, circle_crop, temp_augmentate
import cv2
import matplotlib.pyplot as plt
import shutil

In [2]:
def selectRandomImage():
    import random # Rastgelelik için random kütüphanesini kullanıyoruz
    mylist = list() # Fotoğrafları eklemek için bir liste oluşturuyoruz
    for r, _, f in os.walk(".\\images"): # ./animals/animals klasörü altındaki tüm dosyaları geziyoruz
        if len(f) > 0: # Bulunduğumuz klasördeki  dosya sayısı 0 dan büyükse işlem yapıyoruz
            file = random.choice(f) # Klasör içinden (örneğin kedi klasörü) bir fotoğraf seçiyoruz.
            # Daha sonra bu fotoğrafı ve fotoğraftaki hayvanın adını listemize ekliyoruz.
            mylist.append((f"{r}\\{file}".replace("\\", "/"), f"{r}\\{file}".replace("\\", "/").split("/")[-2]))
    return random.choice(mylist) # Listeden rastgele bir eleman döndürüyoruz.
selectRandomImage()

('./images/3/21177_left.jpeg', '3')

In [11]:
def preprocess(imgPath : str = None, imgClass = None):
    if imgPath == None:
        imgObj = selectRandomImage()
    else:
        imgObj = (imgPath, imgClass)
    #print(imgObj[1])
    img = cv2.imread(imgObj[0])
    img_cropped, flag = crop_image(img)
    height, width, _= img_cropped.shape
    ratio = height/width
    new_sz = 1024
    if width > new_sz:
        new_image = cv2.resize(img_cropped,(new_sz,math.ceil(new_sz*ratio)))  
    else:
        new_image = img_cropped

    circle_cropped = circle_crop(new_image, 30)
    if imgPath == None:
        return (circle_cropped, imgObj[1])
    else:
        return (circle_cropped, imgClass)

# preprocessed_img = preprocess()
# plt.imshow(preprocessed_img)
# _ = cv2.imwrite("./deneme.jpeg", preprocessed_img)

In [4]:
model = keras.models.load_model("0867acc.mdl_wts.hdf5")

In [5]:
def predictRandom():
    ## Bir fotoğrafı çoğaltıp tahmin ettir.
    
    class_names = ["0", "1", "2", "3", "4"]
    img_array, cls = preprocess()
    img_array = tf.expand_dims(img_array, 0) # Veri listesindeki verileri paket halinde hazırlıyoruz.

    predictions = model.predict(img_array) # Bu paketi modelimize veriyoruz ve tahminini yapıyor.
    score = tf.nn.softmax(predictions[0]) # Tahmin sonucunu alıyoruz. Softmax aldığımız değerlerin 0 ile 1 aralığında olmasını sağlıyor


    # Modelin tahminini doğruluk payı ile beraber ekrana bastırıyoruz.
    #print(
    #    "This image most likely belongs to {} with a %{:.2f} confidence."
    #    .format(class_names[np.argmax(score)], 100 * np.max(score))
    #)
    return cls == class_names[np.argmax(score)]
#predictRandom()

In [6]:
def predictMultipleRandom(count):
    t_count = 0
    f_count = 0
    for i in range(count):
        if predictRandom():
            t_count += 1
        else:
            f_count += 1

    print(t_count, f_count, "%"+str((t_count / (t_count + f_count)) * 100))

In [8]:
predictMultipleRandom(100)

63 37 %63.0


In [9]:
def predict(imgPath):
    ## Bir fotoğrafı çoğaltıp tahmin ettir.
    
    class_names = ["0", "1", "2", "3", "4"]
    img_array, cls = preprocess(imgPath, 5)
    img_array = tf.expand_dims(img_array, 0) # Veri listesindeki verileri paket halinde hazırlıyoruz.

    predictions = model.predict(img_array) # Bu paketi modelimize veriyoruz ve tahminini yapıyor.
    score = tf.nn.softmax(predictions[0]) # Tahmin sonucunu alıyoruz. Softmax aldığımız değerlerin 0 ile 1 aralığında olmasını sağlıyor


    # Modelin tahminini doğruluk payı ile beraber ekrana bastırıyoruz.
    #print(
    #    "This image most likely belongs to {} with a %{:.2f} confidence."
    #    .format(class_names[np.argmax(score)], 100 * np.max(score))
    #)
    return class_names[np.argmax(score)]